In [53]:
import urllib2, sys
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from IPython.core.display import HTML
import requests
import time

In [2]:
site= "https://www.hltv.org/results?content=demo&team=7010&startDate=2016-09-01&endDate=2017-09-14"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

C:\Users\mckak\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\mckak\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
matches = soup.find_all("div", class_='results-all')
results = matches[0].find_all("a", class_="a-reset")
urls1 = [result['href'] for result in results]

In [4]:
site= "https://www.hltv.org/results?offset=100&startDate=2016-09-01&endDate=2017-09-14&content=demo&team=7010"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [5]:
matches = soup.find_all("div", class_='results-all')
results = matches[0].find_all("a", class_="a-reset")
urls2 = [result['href'] for result in results]

In [6]:
urls = urls1 + urls2

In [68]:
def parse_match(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    site = 'https://www.hltv.org' + site
    req = urllib2.Request(site,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    
    req = urllib2.Request(site,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)

    demo_url = 'https://www.hltv.org' + soup.find_all("a", class_="flexbox left-right-padding")[0]['href']
    
    try:
        vetos = soup.find_all("div", class_="standard-box veto-box")[1].find_all("div")[0].find_all("div")
        vetos = [veto.text for veto in vetos]
    except:
        vetos = None
        
    stats_url = 'https://www.hltv.org' + [a_element['href'] for a_element in soup.find_all("a") if a_element.text == "Detailed stats"][0]

    tables = soup.find_all("div", class_ = "stats-content", id="all-content")[0]
    tables = tables.find_all("table")
    r = requests.get(site, headers=hdr)
    tables = pd.read_html(r.text, header=0)
    team_a, team_b = tables[0], tables[1]

    return {
        'url': site,
        'vetos': vetos,
        'stats_url': stats_url,
        'teams': [team_a, team_b]
    }

In [ ]:
matches = []
for idx, url in enumerate(urls):
    matches.append(parse_match(url))
    time.sleep(5)
    print 'match {0} done'.format(idx)